<a href="https://colab.research.google.com/github/TurkuNLP/Text_Mining_Course/blob/master/simstring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simstring

## Installation

* simstring needs a little effort to get running
* the necessary steps to follow are here: https://github.com/chokkan/simstring/issues/27
* that will give you a file called `simstring-1.1-cp38-cp38-linux_x86_64.whl`
* install it

In [6]:
%%bash
git clone https://github.com/chokkan/simstring.git


Cloning into 'simstring'...


In [13]:
%%bash
apt install automake swig
cd simstring
./autogen.sh

Reading package lists...
Building dependency tree...
Reading state information...
automake is already the newest version (1:1.15.1-3ubuntu2).
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 29 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,289 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 161260 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpacking swig (3.0.12-1)



aclocal: warning: autoconf input should be named 'configure.ac', not 'configure.in'
automake: warning: autoconf input should be named 'configure.ac', not 'configure.in'
configure.in:31: warning: AM_INIT_AUTOMAKE: two- and three-arguments forms are deprecated.  For more info, see:
configure.in:31: http://www.gnu.org/software/automake/manual/automake.html#Modernize-AM_005fINIT_005fAUTOMAKE-invocation
configure.in:110: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
frontend/Makefile.am:13: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
automake: warning: autoconf input should be named 'configure.ac', not 'configure.in'
configure.in:110: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
sample/Makefile.am:10: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
configure.in:110: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')


In [14]:
%%bash
cd simstring/
./autogen.sh
./configure
make
cd swig/python
./prepare.sh --swig
pip3 install --upgrade pip setuptools wheel


checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking whether gcc understands -c and -o together... yes
checking how to run the C preprocessor... gcc -E
checking for grep that handles long lines and -e... /bin/grep
checking for egrep... /bin/grep -E
checking for ANSI C header files... yes
checking for sys/types.h... yes
checking for sys/stat.h... yes
checking for stdlib.h... yes
checking for string.h... yes
checking for memory.h... yes
checking for strings.h... yes
checking for inttypes.h... yes
checking for stdint.h... yes
checking fo

aclocal: warning: autoconf input should be named 'configure.ac', not 'configure.in'
automake: warning: autoconf input should be named 'configure.ac', not 'configure.in'
configure.in:31: warning: AM_INIT_AUTOMAKE: two- and three-arguments forms are deprecated.  For more info, see:
configure.in:31: http://www.gnu.org/software/automake/manual/automake.html#Modernize-AM_005fINIT_005fAUTOMAKE-invocation
configure.in:110: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
frontend/Makefile.am:13: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
automake: warning: autoconf input should be named 'configure.ac', not 'configure.in'
configure.in:110: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
sample/Makefile.am:10: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
configure.in:110: warning: 'INCLUDES' is the old name for 'AM_CPPFLAGS' (or '*_CPPFLAGS')
ln: failed to create symbolic link './export.cpp': 

In [15]:
%%bash
cd simstring/swig/python
python3 setup.py bdist_wheel

running bdist_wheel
running build
running build_py
creating build
creating build/lib.linux-x86_64-3.7
copying simstring.py -> build/lib.linux-x86_64-3.7
running build_ext
building '_simstring' extension
creating build/temp.linux-x86_64-3.7
g++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/content/simstring/include -I/usr/include/python3.7m -c export.cpp -o build/temp.linux-x86_64-3.7/export.o
g++ -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-a56wZI/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=f

In file included from export.cpp:8:0:
/content/simstring/include/simstring/simstring.h: In member function ‘bool simstring::reader::open(const string&)’:
/content/simstring/include/simstring/simstring.h:838:18: warning: variable ‘num_entries’ set but not used [-Wunused-but-set-variable]
         uint32_t num_entries, max_size;
                  ^~~~~~~~~~~
/content/simstring/include/simstring/simstring.h: In member function ‘bool simstring::reader::check(const string_type&, double)’:
/content/simstring/include/simstring/simstring.h:1020:50: warning: typedef ‘char_type’ locally defined but not used [-Wunused-local-typedefs]
         typedef typename string_type::value_type char_type;
                                                  ^~~~~~~~~
export.cpp: In destructor ‘virtual writer::~writer()’:
export.cpp:112:41: warning: throw will always call terminate() [-Wterminate]
         throw std::runtime_error(message);
                                         ^
export.cpp:112:41: note: in C

In [16]:
!pip install simstring/swig/python/dist/simstring-1.1-cp37-cp37m-linux_x86_64.whl

Processing ./simstring/swig/python/dist/simstring-1.1-cp37-cp37m-linux_x86_64.whl


In [17]:
import simstring
help(simstring)

Help on module simstring:

NAME
    simstring

DESCRIPTION
    # This file was automatically generated by SWIG (http://www.swig.org).
    # Version 3.0.12
    #
    # Do not make changes to this file unless you know what you are doing--modify
    # the SWIG interface file instead.

CLASSES
    builtins.object
        StringVector
        SwigPyIterator
        reader
        writer
    
    class StringVector(builtins.object)
     |  StringVector(*args)
     |  
     |  Methods defined here:
     |  
     |  __bool__(self)
     |  
     |  __del__ lambda self
     |  
     |  __delitem__(self, *args)
     |  
     |  __delslice__(self, i, j)
     |  
     |  __getattr__ lambda self, name
     |  
     |  __getitem__(self, *args)
     |  
     |  __getslice__(self, i, j)
     |  
     |  __init__(self, *args)
     |      Initialize self.  See help(type(self)) for accurate signature.
     |  
     |  __iter__(self)
     |  
     |  __len__(self)
     |  
     |  __nonzero__(self)
     | 

# Usage

1. Index a number of strings
2. Use the index for lookup

In [20]:
import bz2
names=bz2.open("names.txt.bz2","rt").read().splitlines()
print(names)

['Mikael Agricola', 'Adolf Ehrnrooth', 'Tarja Halonen', 'Urho Kekkonen', 'Aleksis Kivi', 'Elias Lönnrot', 'Kalevala', 'C. G. E. Mannerheim', 'Risto Ryti', 'Jean Sibelius', 'Arvo Ylppö', 'Matti Nykänen', 'Väinö Myllyrinne', 'Ville Valo', 'Lalli', 'Väinö Linna', 'Linus Torvalds', 'Spede Pasanen', 'Pentti Linkola', 'Tove Jansson', 'Veikko Hursti', 'Paavo Nurmi', 'Minna Canth', 'Juho Kusti Paasikivi', 'J. V. Snellman', 'Hertta Kuusinen', 'Arto Saari', 'Miina Sillanpää', 'Väinö Tanner', 'Lucina Hagman', 'Kristfrid Ganander', 'Mika Waltari', 'Mika Häkkinen', 'Alvar Aalto', 'Eugen Schauman', 'Tapio Rautavaara', 'Eino Leino', 'Jaakko Pöyry', 'Otto Wille Kuusinen', 'Juice Leskinen', 'Anders Chydenius', 'Uno Cygnaeus', 'Jari Litmanen', 'Katri Helena Kalaoja', 'Fanni Luukkonen', 'Anneli Jäätteenmäki', 'Karl Fazer', 'K. J. Ståhlberg', 'Mauno Koivisto', 'Helene Schjerfbeck', 'Reino Helismaa', 'Jorma Ollila', 'Lauri Törni', 'Georg Henrik von Wright', 'Arndt Pekurinen', 'Tauno Palo', 'Akseli Gallen-K

In [21]:
# index
import os
os.makedirs("test_names",exist_ok=True) #produces a lot of files, make dir
db=simstring.writer("test_names/test_names.db") 
for name in names:
    db.insert(name)
db.close()


In [22]:
db=simstring.reader("test_names/test_names.db")
db.measure=simstring.cosine
db.threshold=0.5
print(db.retrieve("Karita Matila"))
print(db.retrieve("Arvi Lund")) #short name, similarity decreases steeply!

('Karita Mattila',)
('Arvi Lind',)


In [23]:
db.retrieve("Kaarlo Juho Stahlberg")

()

In [24]:
db.retrieve("Kaarlo Juho Ståhlberg")

('K. J. Ståhlberg',)

* Many limitations shown above
* Generic string matching needs further customization
* simstring is a fast first filter for more costly techniques

# Larger index

* all Finnish wikidata strings

In [ ]:
from tqdm import tqdm #progress-bar
os.makedirs("wikidata.db",exist_ok=True)
db=simstring.writer("wikidata.db/wikidata.db")
with open("wikidata.fi") as f:
    for line in tqdm(f):
        line=line.strip()
        # 4-col file with string, two urls, and official label
        # let us index the strings
        s,url1,url2,label=line.split("\t")
        db.insert(s)
db.close()

3995911it [01:58, 33791.00it/s]


* 4M strings - 2min index time

In [ ]:
db=simstring.reader("wikidata.db/wikidata.db")
db.retrieve("Tarja Halonen")

('Ona Halonen',
 'Esa Halonen',
 'Mia Halonen',
 'Tarja Halonen',
 'Tarja Salonen',
 'Tuija Halonen',
 'Maija Halonen',
 'Luokka:Tarja Halonen',
 'Tarja Kaarina Halonen')

In [ ]:
db.retrieve("keskusrikospoliisi")

('rikospoliisi',
 'keskusrikospoliisi',
 'Keskusrikospoliisi',
 'Keskusrikospoliisi',
 'Keskusrikospoliisin Rikosmuseo')

## Lookup speed

In [ ]:
import os
from tqdm import tqdm
with open("wikidata.fi") as f:
    for i,line in enumerate(tqdm(f)):
        line=line.strip()
        db.retrieve(line)
        if i==10000:
            break


10000it [00:10, 927.73it/s]
